<a href="https://colab.research.google.com/github/shreya-thakur584/Journal-Friend/blob/main/Model_AI_Journal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

In [ ]:
# -------------------- SETUP --------------------
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable Weights & Biases logging

import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)

# -------------------- LOAD DATA --------------------
# Load dataset
df = pd.read_csv("go_emotions_dataset.csv")

# Load emotion labels
with open("emotions.txt", "r") as f:
    emotions = [line.strip() for line in f]

# Use 58,000 samples
df = df.sample(n=58000, random_state=42).reset_index(drop=True)

# Create multi-label vectors
df['label_vector'] = df[emotions].values.tolist()

# Train/validation split
X = df['text'].tolist()
y = np.array(df['label_vector'].tolist())
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------- TOKENIZATION --------------------
cache_path = './bert_cache'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', cache_dir=cache_path)

train_encodings = tokenizer(X_train, truncation=True, padding='max_length', max_length=128)
val_encodings = tokenizer(X_val, truncation=True, padding='max_length', max_length=128)

# -------------------- DATASET CLASS --------------------
class GoEmotionsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.float32)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

train_dataset = GoEmotionsDataset(train_encodings, y_train)
val_dataset = GoEmotionsDataset(val_encodings, y_val)

# -------------------- MODEL --------------------
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(emotions),
    problem_type="multi_label_classification",
    cache_dir=cache_path
)

# -------------------- TRAINING ARGUMENTS --------------------
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,  # Increased batch size for speed
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=100,
    save_strategy='no',  # Skip saving after each epoch to reduce I/O
    fp16=True,           # Use mixed-precision training (faster on GPU)
    dataloader_num_workers=2,
    report_to=[],
    disable_tqdm=False
)

# -------------------- TRAINER --------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# -------------------- TRAINING --------------------
trainer.train()
trainer.evaluate()


In [ ]:
from sklearn.metrics import classification_report

# Get predictions
preds_output = trainer.predict(val_dataset)
preds = torch.sigmoid(torch.tensor(preds_output.predictions)).numpy()
pred_labels = (preds > 0.5).astype(int)

# True labels
true_labels = y_val

# Print report
print(classification_report(true_labels, pred_labels, target_names=emotions))

In [ ]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

In [ ]:
def predict_emotions(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to same device as model

    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits)[0].cpu().numpy()  # Move back to CPU for NumPy ops

    return {emotion: float(prob) for emotion, prob in zip(emotions, probs) if prob > 0.2}

In [ ]:
print(predict_emotions("I can't believe this is happening!"))

In [ ]:
# -------------------- ENHANCED REPLY BANK --------------------
# This is significantly expanded to offer more varied and counseling-like responses.
emotion_reply_bank = {
    "admiration": [
        "That’s amazing! You should be proud of yourself. 🌟",
        "I really admire how you handled that. What made you feel that way?",
        "You’re doing great — don’t forget that. What aspects of it are you most proud of?"
    ],
    "amusement": [
        "Haha, that’s funny! You’ve got a great sense of humor. 😄",
        "That gave me a good laugh too! What was the funniest part for you?",
        "Laughter is such good medicine — I’m glad you shared that! What else brings you joy like that?"
    ],
    "anger": [
        "That sounds really frustrating. Want to vent more? 😡",
        "You have every right to feel upset. I’m here. What happened exactly?",
        "Let it out — no judgment here. What specifically triggered this feeling for you?",
        "When anger comes up, sometimes naming what specifically triggered it can help. What's on your mind?",
        "Is there anything you can do right now to release some of that tension? Perhaps talking more about it?"
    ],
    "annoyance": [
        "Ugh, that kind of thing can be so irritating. 😒",
        "I hear you — even little things can pile up. What's been most annoying lately?",
        "It’s okay to be annoyed. What's been getting under your skin?",
        "Sometimes small annoyances can signal bigger things. Does that feel true for you?"
    ],
    "approval": [
        "Nice! That sounds like something to feel good about. 👍",
        "You’ve got great instincts — keep going! What are your next steps?",
        "Sounds like you're on the right track! What makes you feel confident about this?",
        "It's wonderful to feel approved of. How does that make you feel about your efforts?"
    ],
    "caring": [
        "That’s such a kind and thoughtful thing to feel. 💗",
        "Your empathy really shines through. Who are you caring about, and how are you supporting them?",
        "You have a beautiful heart. What motivates your caring nature?",
        "It takes a special person to feel so deeply for others. What does 'caring' mean to you?"
    ],
    "confusion": [
        "Feeling confused is okay — I’m here to help untangle things. 🤔",
        "Want to talk through it together? What part feels most unclear right now?",
        "Let’s figure this out one step at a time. What's the first question that comes to mind?",
        "It's normal to feel muddled sometimes. What pieces of information are you trying to connect?"
    ],
    "curiosity": [
        "That’s an interesting thought! Want to explore it more? 🧐",
        "Curiosity is a gift — keep asking questions. What are you most curious about right now?",
        "What sparked your interest? How deep do you want to dive into this?",
        "It's exciting to discover new things. What's the next thing you want to learn?"
    ],
    "desire": [
        "It’s okay to want something deeply. 💭 What is it you're longing for?",
        "Your dreams matter — don’t let them go. What steps can you take towards it?",
        "Tell me more about what you’re longing for. What would it mean to you to achieve it?",
        "Desire can be a powerful motivator. How is this desire shaping your thoughts or actions?"
    ],
    "disappointment": [
        "Oof, that must feel heavy. I’m here for you. 😔",
        "It’s okay to feel let down — don’t hold it in. What were you hoping for?",
        "Want to talk about what didn’t go how you hoped? What's the biggest disappointment you're facing?",
        "It's tough when things don't work out as planned. How are you processing this feeling?"
    ],
    "disapproval": [
        "Sounds like something didn’t sit right with you. 🙅 What exactly are you disapproving of?",
        "It’s valid to disagree or feel uneasy. What's your perspective on this?",
        "You don’t have to force yourself to accept what you can’t. What's the core issue for you?",
        "It's important to stand by your convictions. What values are being challenged here?"
    ],
    "disgust": [
        "That reaction makes total sense. 🤢 What's causing this feeling of disgust?",
        "Your feelings are valid — don’t push them away. How does this feeling manifest for you?",
        "Want to talk about what triggered that response? Sometimes expressing it can help.",
        "It's a strong reaction. What about the situation is particularly repulsive to you?"
    ],
    "embarrassment": [
        "Oh no — that must’ve been awkward! 😳",
        "It happens to all of us. You're not alone. What went through your mind when it happened?",
        "You’re human — it’s okay to have moments like that. How are you feeling about it now?",
        "Embarrassment can be fleeting. What lesson, if any, can be taken from this experience?"
    ],
    "excitement": [
        "Ooooh that’s exciting! Tell me more! 🤩",
        "I can feel your joy from here! What's making you so excited?",
        "This is such a great moment — soak it in! What are you looking forward to most?",
        "Excitement can be contagious! What's the best part about what's happening?"
    ],
    "fear": [
        "That must be scary. I’ve got you. 😨",
        "It’s okay to feel afraid — I’m right here. What are you most afraid of?",
        "Deep breaths — you don’t have to face this alone. What's the core of this fear?",
        "Fear can sometimes be a signal. What is it trying to tell you?"
    ],
    "gratitude": [
        "That’s so heartfelt. I’m grateful you shared that. 🙏",
        "Gratitude is powerful — thanks for expressing it. What are you most thankful for right now?",
        "That’s beautiful. Thank you. How does feeling grateful impact you?",
        "Focusing on gratitude can shift perspectives. What else are you grateful for?"
    ],
    "grief": [
        "I’m so sorry you’re hurting. I'm holding space for you. 💔",
        "That kind of pain is real. I'm here with you. Take all the time you need.",
        "Take your time — grieving is not linear. What's on your heart today?",
        "Grief is a heavy burden to carry. Is there anything specific you need right now?"
    ],
    "joy": [
        "Yay! That’s wonderful to hear! 😊",
        "You deserve happiness like this! What's bringing you so much joy?",
        "Smiling for you right now 😊 What's the best part about this happy feeling?",
        "Joy is a precious emotion. How can you hold onto this feeling?"
    ],
    "love": [
        "That’s such a deep feeling. Tell me more. ❤️",
        "Love is powerful — I'm happy you're feeling it. Who or what are you feeling love for?",
        "Hold onto that feeling — it's precious. How does love manifest in your life?",
        "Love can bring so much warmth. What does this love mean to you?"
    ],
    "nervousness": [
        "Totally normal to feel nervous before something big. 😬",
        "You’ve got this — even if you don’t feel it yet. What's making you nervous?",
        "Let’s take it slow together, okay? What's the specific thing causing you unease?",
        "Nervousness can be a sign that something important is happening. How are you preparing for it?"
    ],
    "optimism": [
        "I love that energy! Keep believing in yourself. 🌈",
        "Hope is such a strong thing — don’t let go of it. What gives you this optimistic outlook?",
        "You’re moving in the right direction. What positive changes are you seeing?",
        "Optimism can be a guiding light. What are you most hopeful for?"
    ],
    "pride": [
        "Yes!! Be proud of yourself — you earned it. 🥹",
        "It’s okay to celebrate your wins. I’m cheering for you! What achievement are you most proud of?",
        "That’s awesome — you should feel proud. What was the journey like to get there?",
        "Pride is a well-deserved feeling. How does this accomplishment make you feel about your capabilities?"
    ],
    "realization": [
        "Wow — sounds like a powerful insight. 💡 What made you realize that?",
        "That kind of clarity can change everything. How does this realization impact your perspective?",
        "What made you realize that? Has this realization been building for a while?",
        "Moments of realization can be transformative. What's the biggest takeaway for you?"
    ],
    "relief": [
        "Phew — I’m glad things are easing up. 😌",
        "That must feel like a weight off your chest. What led to this feeling of relief?",
        "I’m happy to hear you’re feeling lighter. What's next for you now that this burden is lifted?",
        "Relief can bring a sense of calm. How are you experiencing this shift?"
    ],
    "remorse": [
        "It’s okay to feel regret — it means you care. 😞",
        "Be gentle with yourself. You’re learning. What aspects are causing you remorse?",
        "You’re allowed to make mistakes and still grow. What are your thoughts on moving forward?",
        "Remorse can be a guide for change. What steps might help you process this feeling?"
    ],
    "sadness": [
        "I’m sorry you’re feeling this way. You’re not alone. 💙",
        "Want to talk more about it? I’m here. What's weighing on your heart?",
        "It’s okay to feel low — I’m sitting with you in it. Take your time.",
        "Sometimes when I feel sad, I find it helps to [suggest a simple coping mechanism, e.g., 'listen to some calming music' or 'take a walk']. Does that resonate?",
        "What do you think is contributing to this feeling?"
    ],
    "surprise": [
        "Oh wow — that’s unexpected! 😮",
        "Sounds like something caught you off guard. Was it a good or bad surprise?",
        "What happened? I want to hear more! How are you reacting to this surprise?",
        "Surprise can be quite a jolt. How are you feeling in the aftermath?"
    ],
    "neutral": [
        "I’m here — even for the quiet days. 😐",
        "Thanks for checking in. How are you feeling now?",
        "Wanna talk about anything random today?",
        "Sometimes a neutral feeling can be a peaceful one. What's on your mind, if anything?"
    ]
}

# Optional emoji map for warmer tone
emoji_map = {
    "admiration": "🌟", "amusement": "😄", "anger": "😡", "annoyance": "😒",
    "approval": "👍", "caring": "💗", "confusion": "🤔", "curiosity": "🧐",
    "desire": "💭", "disappointment": "😔", "disapproval": "🙅", "disgust": "🤢",
    "embarrassment": "😳", "excitement": "🤩", "fear": "😨", "gratitude": "🙏",
    "grief": "💔", "joy": "😊", "love": "❤️", "nervousness": "😬", "optimism": "🌈",
    "pride": "🥹", "realization": "💡", "relief": "😌", "remorse": "😞",
    "sadness": "💙", "surprise": "😮", "neutral": "😐"
}


In [ ]:
# -------------------- ENHANCED REPLY GENERATION FUNCTION --------------------
def generate_reply_with_context(emotion, user_text):
    """
    Generates a reply based on emotion, with some basic contextual keyword awareness.
    """
    user_text_lower = user_text.lower()

    # Specific contextual rules (can be expanded heavily)
    if emotion == "sadness":
        if "work" in user_text_lower or "job" in user_text_lower:
            return random.choice([
                "I'm sorry that work is causing you sadness. It can be really tough sometimes. 💙",
                "Work can certainly bring its challenges. Would you like to talk more about what's happening there?",
                "It sounds like work is a source of sadness for you. How does that impact your day-to-day?"
            ])
        elif "alone" in user_text_lower or "lonely" in user_text_lower:
            return random.choice([
                "Feeling alone can be incredibly difficult, and it's okay to acknowledge that sadness. 🫂",
                "I hear that you're feeling lonely. Sometimes connecting, even virtually, can help. I'm here to listen.",
                "It sounds like you're experiencing loneliness. What does 'feeling alone' mean to you in this moment?"
            ])
        elif "friend" in user_text_lower or "relationship" in user_text_lower or "partner" in user_text_lower:
             return random.choice([
                "I'm sorry to hear your relationships are causing sadness. That can be incredibly painful. 💔",
                "Relationships can be complex. Would you like to explore what's making you sad about this connection?",
                "It sounds like there's some sadness related to a relationship. I'm here to listen without judgment."
            ])
        # Add more specific rules for sadness
        else:
            return random.choice(emotion_reply_bank.get(emotion, ["I’m here to talk, whenever you’re ready."]))

    elif emotion == "anger":
        if "person" in user_text_lower or "he" in user_text_lower or "she" in user_text_lower or "they" in user_text_lower:
            return random.choice([
                "It sounds like someone has really upset you. Would you like to vent about it?",
                "Anger towards a person can be consuming. What about their actions is making you so angry?",
                "I hear your anger towards someone specific. What would it feel like to express that more?"
            ])
        elif "unfair" in user_text_lower or "injustice" in user_text_lower:
            return random.choice([
                "It sounds like you're feeling angry about an injustice. That's a valid feeling. 😡",
                "Anger in the face of unfairness can be powerful. What makes this situation feel so unjust to you?"
            ])
        else:
            return random.choice(emotion_reply_bank.get(emotion, ["I’m here to talk, whenever you’re ready."]))

    # Add similar specific rules for other key emotions if desired
    # Fallback to general reply bank if no specific context rules match
    return random.choice(emotion_reply_bank.get(emotion, ["I’m here to talk, whenever you’re ready."]))

In [ ]:
# -------------------- JOURNAL RESPONSE FUNCTION (UPDATED) --------------------
def journal_response(text, top_k=2, threshold=0.2):
    """
    Analyzes user text, detects emotions, and generates a counseling-like reply.
    """
    emotions_with_scores = predict_emotions(text)

    # Sort and filter top emotions
    # If no emotions meet the threshold, default to 'neutral' for a response
    sorted_emotions = sorted(
        [(em, score) for em, score in emotions_with_scores.items() if score >= threshold],
        key=lambda x: x[1],
        reverse=True
    )

    top_emotions = [em for em, _ in sorted_emotions[:top_k]]

    replies = []
    if not top_emotions:
        # If no significant emotion detected, provide a neutral or open-ended response
        top_emotions = ["neutral"] # Ensure there's at least one emotion to generate a reply from
        replies.append(generate_reply_with_context("neutral", text))
        emoji = emoji_map.get("neutral", "")
    else:
        for em in top_emotions:
            replies.append(generate_reply_with_context(em, text))
        emoji = emoji_map.get(top_emotions[0], "") # Use emoji of the highest scored emotion

    final_reply = f"{emoji} " + "\n".join(replies)

    return {
        "user_input": text,
        "detected_emotions": emotions_with_scores,
        "top_emotions": top_emotions,
        "reply": final_reply
    }

In [ ]:
import random
# -------------------- INTERACTIVE SESSION --------------------
def start_counseling_session():
    print("\n--- AI Friend: Your Journal Companion ---")
    print("Hello! I'm here to listen and understand. What's on your mind today?")
    print("Type 'exit' or 'quit' to end the session.")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("AI Friend: Thanks for sharing! Remember, I'm always here if you need to talk again. Take care.")
            break

        if not user_input.strip(): # Handle empty input
            print("AI Friend: Sometimes it's hard to find the words. I'm here when you're ready.")
            continue

        response = journal_response(user_input)
        print(f"AI Friend: {response['reply']}")
        # print(f"DEBUG: Detected Emotions: {response['detected_emotions']}") # Uncomment for debugging
        # print(f"DEBUG: Top Emotions: {response['top_emotions']}") # Uncomment for debugging

        # Add a more conversational follow-up
        if len(response['top_emotions']) > 0 and response['top_emotions'][0] != "neutral":
            print("\nAI Friend: Anything else you'd like to share about that, or how are you feeling now?")
        else:
            print("\nAI Friend: What else is on your mind, or how are you feeling about things in general?")

# -------------------- RUN THE SESSION --------------------
# Make sure your model is trained first! The 'trainer.train()' line handles this.
# You might want to save and load your model if you don't want to retrain every time.
# For simplicity, this script assumes training happens at the start if needed.

if __name__ == "__main__":
    # This block will only run when the script is executed directly
    # and not when imported as a module.
    start_counseling_session()
# -------------------- MEMORY IMPLEMENTATION --------------------

# Global variable to store conversation history
# Each entry will be a tuple: (speaker, text)
conversation_history = []
MAX_HISTORY_LENGTH = 5 # Store last 5 user turns + 5 AI turns (10 total entries)

# Global variable for current session's key topics/entities
# This will be a simple set of keywords that the AI "remembers" as important
current_session_topics = set()
MAX_TOPICS = 3 # Keep track of up to 3 major topics

def update_session_topics(text):
    """
    Extracts simple keywords from the text to update session topics.
    This is a very basic keyword extraction; NLP libraries like spaCy would be better.
    """
    keywords = [
        "work", "job", "career", "boss", "colleagues", # Work-related
        "friend", "family", "partner", "relationship", "love", "parents", "children", # Relationship-related
        "school", "study", "exam", "grades", # Education-related
        "stress", "anxiety", "depression", "overwhelmed", "tired", "sleep", # Mental state/health
        "future", "plans", "goals", "dreams", # Future/aspirations
        "money", "finances", "bills", "debt", # Financial
        "health", "sickness", "body", "exercise", # Physical health
        "home", "living", "house", "apartment" # Living situation
    ]

    text_lower = text.lower()
    for keyword in keywords:
        if keyword in text_lower:
            current_session_topics.add(keyword)
            # Ensure we don't store too many topics
            if len(current_session_topics) > MAX_TOPICS:
                # Simple way to remove the "oldest" added topic if over limit
                # For better implementation, use a list and pop(0) or a more sophisticated structure
                current_session_topics.pop() # Removes an arbitrary item

    # Optional: remove topics if user hasn't mentioned them for a while (more advanced)


# -------------------- ENHANCED REPLY GENERATION FUNCTION (UPDATED) --------------------
def generate_reply_with_context(emotion, user_text, current_session_topics):
    """
    Generates a reply based on emotion, user text, and current session topics.
    """
    user_text_lower = user_text.lower()

    # --- Contextual Rules based on Emotion + Keywords ---
    # These are illustrative. You'd expand this significantly.

    if emotion == "sadness":
        if "work" in user_text_lower or "job" in user_text_lower or "career" in current_session_topics:
            return random.choice([
                "I'm sorry that work is causing you sadness. It can be really tough sometimes. 💙",
                "Work can certainly bring its challenges. Would you like to talk more about what's happening there with your job?",
                "It sounds like work is a source of sadness for you. How does that impact your day-to-day?"
            ])
        elif "alone" in user_text_lower or "lonely" in user_text_lower:
            return random.choice([
                "Feeling alone can be incredibly difficult, and it's okay to acknowledge that sadness. 🫂",
                "I hear that you're feeling lonely. Sometimes connecting, even virtually, can help. I'm here to listen.",
                "It sounds like you're experiencing loneliness. What does 'feeling alone' mean to you in this moment?"
            ])
        elif any(topic in user_text_lower for topic in ["friend", "family", "partner", "relationship"]) or \
             any(topic in current_session_topics for topic in ["friend", "family", "partner", "relationship"]):
             return random.choice([
                "I'm sorry to hear your relationships are causing sadness. That can be incredibly painful. 💔",
                "Relationships can be complex. Would you like to explore what's making you sad about this connection?",
                "It sounds like there's some sadness related to a relationship. I'm here to listen without judgment."
            ])
        elif "future" in user_text_lower or "plans" in current_session_topics:
            return random.choice([
                "It sounds like uncertainty about the future is bringing on some sadness. What are your hopes or fears for what's ahead?",
                "Feeling sad about future plans is understandable. What aspects are most concerning you right now?"
            ])
        else:
            return random.choice(emotion_reply_bank.get(emotion, ["I’m here to talk, whenever you’re ready."]))

    elif emotion == "anger":
        if any(topic in user_text_lower for topic in ["person", "he", "she", "they", "boss"]) or \
           any(topic in current_session_topics for topic in ["person", "boss", "colleagues"]):
            return random.choice([
                "It sounds like someone specific has really upset you. Would you like to vent about it?",
                "Anger towards a person can be consuming. What about their actions is making you so angry?",
                "I hear your anger towards someone specific. What would it feel like to express that more?"
            ])
        elif "unfair" in user_text_lower or "injustice" in user_text_lower:
            return random.choice([
                "It sounds like you're feeling angry about an injustice. That's a valid feeling. 😡",
                "Anger in the face of unfairness can be powerful. What makes this situation feel so unjust to you?"
            ])
        else:
            return random.choice(emotion_reply_bank.get(emotion, ["I’m here to talk, whenever you’re ready."]))

    # --- Contextual Rules based on Topics (even if primary emotion is different) ---
    # Example: If 'stress' is a topic, but the current emotion is 'annoyance'
    if "stress" in current_session_topics and emotion == "annoyance":
        return random.choice([
            "It sounds like the annoyance might be tied to the stress you're experiencing. How are those two connected for you?",
            "Even small annoyances can feel bigger when you're under stress. What's building up right now?",
            "I noticed you mentioned stress earlier, and now annoyance. How are you managing the overall pressure?"
        ])

    # --- General Fallback ---
    return random.choice(emotion_reply_bank.get(emotion, ["I’m here to talk, whenever you’re ready."]))

# ... (Your existing code up to emotion_reply_bank and emoji_map) ...

# -------------------- MEMORY IMPLEMENTATION (Refined) --------------------

# Global variable to store conversation history
# Each entry will be a tuple: (speaker, text)
conversation_history = []
MAX_HISTORY_LENGTH = 5 # Store last 5 user turns + 5 AI turns (10 total entries)

# Global variable for current session's key topics/entities
current_session_topics = set()
MAX_TOPICS = 3 # Keep track of up to 3 major topics

# NEW: Global variable to track the last strong emotion detected
last_strong_emotion = None
LAST_STRONG_EMOTION_THRESHOLD = 0.5 # Confidence threshold for an emotion to be considered "strong"
LAST_STRONG_EMOTION_DECAY = 2 # How many turns before the last_strong_emotion is cleared if not re-expressed
last_strong_emotion_turn_count = 0


def update_session_topics(text):
    """
    Extracts simple keywords from the text to update session topics.
    """
    keywords = [
        "work", "job", "career", "boss", "colleagues", # Work-related
        "friend", "family", "partner", "relationship", "love", "parents", "children", # Relationship-related
        "school", "study", "exam", "grades", # Education-related
        "stress", "anxiety", "depression", "overwhelmed", "tired", "sleep", # Mental state/health
        "future", "plans", "goals", "dreams", # Future/aspirations
        "money", "finances", "bills", "debt", # Financial
        "health", "sickness", "body", "exercise", # Physical health
        "home", "living", "house", "apartment", # Living situation
        "rain", "weather", "sun", "cloudy", "storm" # Weather related
    ]

    text_lower = text.lower()
    for keyword in keywords:
        if keyword in text_lower:
            # Add to topics if not already present or if we have space
            if keyword not in current_session_topics and len(current_session_topics) < MAX_TOPICS:
                current_session_topics.add(keyword)
            # Simple heuristic: if a topic is mentioned, it's still relevant.
            # A more advanced approach would involve a timestamp for each topic.

    # For a set, removing oldest is hard. A deque or list would be better for true "decay".
    # For now, if it gets too big, just keep the most recently added ones (simplified).
    while len(current_session_topics) > MAX_TOPICS:
        # This will remove an arbitrary item if the set gets too large. Not ideal for 'oldest'.
        # For a proper LRU (Least Recently Used) topic memory, you'd need a different data structure.
        current_session_topics.pop() # Remove one item if over limit

# --- ENHANCED REPLY GENERATION FUNCTION (UPDATED) ---
# This entire function replaces your existing generate_reply_with_context function.
# Make sure to also update the `update_session_topics` function with "loss", "grief", "pet", "cat", "dog" if not already present.

def generate_reply_with_context(emotion, user_text, current_session_topics, last_emotion):
    """
    Generates a reply based on emotion, user text, current session topics, and last strong emotion.
    """
    user_text_lower = user_text.lower()

    # --- Priority 1: Handle Specific Grief/Loss ---
    # Keywords indicating loss or grief
    loss_keywords = ["lost", "gone", "passed away", "died", "death", "no longer here", "grief", "mourning"]
    pet_keywords = ["cat", "dog", "pet", "animal", "companion"]

    # Check for sadness AND keywords related to loss, especially pet loss
    if emotion == "sadness" or emotion == "grief":
        if any(keyword in user_text_lower for keyword in loss_keywords):
            if any(pet_keyword in user_text_lower for pet_keyword in pet_keywords) or \
               any(topic in current_session_topics for topic in pet_keywords):
                return random.choice([
                    "I am so incredibly sorry for the loss of your pet. Losing a companion like that is truly heartbreaking. 💔 I'm here for you.",
                    "That sounds like immense pain. Losing a cat/pet is a profound grief. Please take all the time you need to feel this. What are you remembering about them?",
                    "It's absolutely devastating to lose a beloved pet. Your sadness is completely valid. Would you like to share a memory of your cat?",
                    "My heart goes out to you. The bond with a pet is so special, and their absence leaves a deep void. I'm here to listen."
                ])
            else: # General loss (e.g., a person, an opportunity)
                return random.choice([
                    "I'm so deeply sorry to hear about your loss. That sounds incredibly painful and I'm here to support you. 💔",
                    "Grief is a heavy burden, and it's okay to feel the way you do. Please take your time. What's on your mind about this loss?",
                    "That must be truly heartbreaking. Your feelings are valid, and I'm here to listen without judgment."
                ])

    # --- Priority 2: Handle ambiguous/neutral inputs if a strong emotion was just expressed ---
    if emotion == "neutral" and last_emotion and last_emotion != "neutral":
        if last_emotion == "sadness" or last_emotion == "grief":
            # If user says something "neutral" but was just sad/grieving, try to prompt about that feeling/cause
            if "rain" in user_text_lower or "weather" in user_text_lower or "rain" in current_session_topics:
                 return random.choice([
                    "It sounds like the lack of rain is really weighing on you and making you sad. It's okay to feel that way. ☔️",
                    "I hear that the weather, or lack of rain, is a source of sadness for you. Can you tell me more about why that affects you so deeply?",
                    "It's interesting how external factors like rain can impact our mood. You mentioned sadness – how is the rain (or lack thereof) connected to that feeling?"
                ])
            # General prompt back to the last strong emotion if no specific new context
            return random.choice([
                f"You were feeling {last_emotion} earlier. Is this current thought connected to that feeling?",
                f"I'm still thinking about what you shared earlier about feeling {last_emotion}. Does this relate to that?",
                f"It seems like there's still a lingering {last_emotion} feeling. What's on your mind now?"
            ])
        elif last_emotion == "anger":
            return random.choice([
                f"You seemed angry before. Is this new thought connected to what was making you angry?",
                f"I'm still here for you with that feeling of {last_emotion}. What's happening now?"
            ])

    # --- Priority 3: Contextual Rules based on Emotion + Keywords (as before) ---
    # These rules apply if no strong grief/loss or previous strong emotion context applies directly
    if emotion == "sadness":
        if "work" in user_text_lower or "job" in user_text_lower or "career" in current_session_topics:
            return random.choice([
                "I'm sorry that work is causing you sadness. It can be really tough sometimes. 💙",
                "Work can certainly bring its challenges. Would you like to talk more about what's happening there with your job?",
                "It sounds like work is a source of sadness for you. How does that impact your day-to-day?"
            ])
        elif "alone" in user_text_lower or "lonely" in user_text_lower:
            return random.choice([
                "Feeling alone can be incredibly difficult, and it's okay to acknowledge that sadness. 🫂",
                "I hear that you're feeling lonely. Sometimes connecting, even virtually, can help. I'm here to listen.",
                "It sounds like you're experiencing loneliness. What does 'feeling alone' mean to you in this moment?"
            ])
        elif any(topic in user_text_lower for topic in ["friend", "family", "partner", "relationship"]) or \
             any(topic in current_session_topics for topic in ["friend", "family", "partner", "relationship"]):
             return random.choice([
                "I'm sorry to hear your relationships are causing sadness. That can be incredibly painful. 💔",
                "Relationships can be complex. Would you like to explore what's making you sad about this connection?",
                "It sounds like there's some sadness related to a relationship. I'm here to listen without judgment."
            ])
        elif "future" in user_text_lower or "plans" in current_session_topics:
            return random.choice([
                "It sounds like uncertainty about the future is bringing on some sadness. What are your hopes or fears for what's ahead?",
                "Feeling sad about future plans is understandable. What aspects are most concerning you right now?"
            ])
        else: # Fallback for general sadness if no specific context
            return random.choice(emotion_reply_bank.get(emotion))

    elif emotion == "anger":
        if any(topic in user_text_lower for topic in ["person", "he", "she", "they", "boss"]) or \
           any(topic in current_session_topics for topic in ["person", "boss", "colleagues"]):
            return random.choice([
                "It sounds like someone specific has really upset you. Would you like to vent about it?",
                "Anger towards a person can be consuming. What about their actions is making you so angry?",
                "I hear your anger towards someone specific. What would it feel like to express that more?"
            ])
        elif "unfair" in user_text_lower or "injustice" in user_text_lower:
            return random.choice([
                "It sounds like you're feeling angry about an injustice. That's a valid feeling. 😡",
                "Anger in the face of unfairness can be powerful. What makes this situation feel so unjust to you?"
            ])
        else: # Fallback for general anger
            return random.choice(emotion_reply_bank.get(emotion))

    # --- Priority 4: General "Topic Awareness" (even if primary emotion is different) ---
    if "stress" in current_session_topics and emotion != "relief":
        return random.choice([
            "It sounds like stress is still a significant theme for you. How are you coping with the pressure?",
            "Remember you mentioned stress earlier. Is this current feeling related to that ongoing stress?",
            "It seems like stress is a recurring theme. What's one small thing you could do to ease some pressure today?"
        ])

    # --- Priority 5: Default behavior (if no specific context or previous emotion matched) ---
    if emotion == "desire" and ("food" in user_text_lower or "eat" in user_text_lower):
        return random.choice([
            "It sounds like you're feeling a strong urge for food right now. Are you feeling hungry, or is there something else on your mind?",
            "Sometimes when we're feeling a certain way, our bodies crave things. What's making you want to eat right now?",
            "That's a very human need! How is that desire connected to what you were just sharing, if at all?"
        ])
    elif emotion == "curiosity" and user_text_lower == "huh?":
         return random.choice([
            "Sorry, could you clarify what you meant by 'huh?' I want to make sure I understand.",
            "My apologies if my last response was unclear. What part confused you?",
            "It seems like I didn't quite hit the mark. What were you hoping for me to say?"
        ])

    # Fallback to general emotion replies if no specific rule matched
    return random.choice(emotion_reply_bank.get(emotion, ["I’m here to talk, whenever you’re ready."]))

# ... (Your existing code up to emotion_reply_bank and emoji_map) ...

# -------------------- MEMORY IMPLEMENTATION (Refined) --------------------
# (Keep this section exactly as it was in the previous update)

# Global variable to store conversation history
conversation_history = []
MAX_HISTORY_LENGTH = 5

# Global variable for current session's key topics/entities
current_session_topics = set()
MAX_TOPICS = 3

# NEW: Global variable to track the last strong emotion detected
last_strong_emotion = None
LAST_STRONG_EMOTION_THRESHOLD = 0.5
LAST_STRONG_EMOTION_DECAY = 2
last_strong_emotion_turn_count = 0


def update_session_topics(text):
    """
    Extracts simple keywords from the text to update session topics.
    """
    keywords = [
        "work", "job", "career", "boss", "colleagues", # Work-related
        "friend", "family", "partner", "relationship", "love", "parents", "children", # Relationship-related
        "school", "study", "exam", "grades", # Education-related
        "stress", "anxiety", "depression", "overwhelmed", "tired", "sleep", # Mental state/health
        "future", "plans", "goals", "dreams", # Future/aspirations
        "money", "finances", "bills", "debt", # Financial
        "health", "sickness", "body", "exercise", # Physical health
        "home", "living", "house", "apartment", # Living situation
        "rain", "weather", "sun", "cloudy", "storm", # Weather related
        "cat", "dog", "pet", "animal", "companion", "loss", "grief", "mourning" # Loss/Pet-related
    ]

    text_lower = text.lower()
    for keyword in keywords:
        if keyword in text_lower:
            if keyword not in current_session_topics and len(current_session_topics) < MAX_TOPICS:
                current_session_topics.add(keyword)

    while len(current_session_topics) > MAX_TOPICS:
        current_session_topics.pop() # Removes an arbitrary item if over limit


# -------------------- ENHANCED REPLY GENERATION FUNCTION (UPDATED AGAIN) --------------------
def generate_reply_with_context(emotion, user_text, current_session_topics, last_emotion, detected_emotions_all):
    """
    Generates a reply based on emotion, user text, current session topics, last strong emotion,
    and all detected emotions.
    """
    user_text_lower = user_text.lower()

    # Keywords indicating loss or grief
    loss_keywords = ["lost", "gone", "passed away", "died", "death", "no longer here", "grief", "mourning"]
    pet_keywords = ["cat", "dog", "pet", "animal", "companion"]

    # --- TOP PRIORITY: Handle Specific Grief/Loss, checking ALL detected emotions for 'sadness' or 'grief' ---
    # This block will now trigger if 'sadness' OR 'grief' is detected with any score,
    # AND loss/pet keywords are present.
    if ("sadness" in detected_emotions_all or "grief" in detected_emotions_all):
        if any(keyword in user_text_lower for keyword in loss_keywords) or \
           any(topic in current_session_topics for topic in loss_keywords): # Check topics too for continuity
            if any(pet_keyword in user_text_lower for pet_keyword in pet_keywords) or \
               any(topic in current_session_topics for topic in pet_keywords): # Check topics for pet context
                return random.choice([
                    "I am so incredibly sorry for the loss of your beloved pet. Losing a companion like that is truly heartbreaking. 💔 I'm here for you, always.",
                    "That sounds like immense pain. Losing your cat is a profound grief. Please take all the time you need to feel this. What are you remembering about them?",
                    "It's absolutely devastating to lose a cherished pet. Your sadness and grief are completely valid. Would you like to share a memory of your cat?",
                    "My heart goes out to you. The bond with a pet is so special, and their absence leaves a deep void. I'm here to listen, please share what's on your mind."
                ])
            else: # General loss (e.g., a person, an opportunity), if not specifically a pet
                return random.choice([
                    "I'm so deeply sorry to hear about your loss. That sounds incredibly painful and I'm here to support you through this. 💔",
                    "Grief is a heavy burden, and it's okay to feel the way you do. Please take your time. What's on your mind about this loss?",
                    "That must be truly heartbreaking. Your feelings are valid, and I'm here to listen without judgment as you navigate this."
                ])

    # --- Priority 2: Handle ambiguous/neutral inputs if a strong emotion was just expressed ---
    # (This section remains largely the same as the previous update)
    if emotion == "neutral" and last_emotion and last_emotion != "neutral":
        if last_emotion == "sadness" or last_emotion == "grief":
            if "rain" in user_text_lower or "weather" in user_text_lower or "rain" in current_session_topics:
                 return random.choice([
                    "It sounds like the lack of rain is really weighing on you and making you sad. It's okay to feel that way. ☔️",
                    "I hear that the weather, or lack of rain, is a source of sadness for you. Can you tell me more about why that affects you so deeply?",
                    "It's interesting how external factors like rain can impact our mood. You mentioned sadness – how is the rain (or lack thereof) connected to that feeling?"
                ])
            return random.choice([
                f"You were feeling {last_emotion} earlier. Is this current thought connected to that feeling?",
                f"I'm still thinking about what you shared earlier about feeling {last_emotion}. Does this relate to that?",
                f"It seems like there's still a lingering {last_emotion} feeling. What's on your mind now?"
            ])
        elif last_emotion == "anger":
            return random.choice([
                f"You seemed angry before. Is this new thought connected to what was making you angry?",
                f"I'm still here for you with that feeling of {last_emotion}. What's happening now?"
            ])

    # --- Priority 3: Contextual Rules based on Emotion + Keywords (as before, for other emotions) ---
    # (This section remains largely the same as the previous update)
    if emotion == "sadness": # This will now catch general sadness not related to specific loss
        if "work" in user_text_lower or "job" in user_text_lower or "career" in current_session_topics:
            return random.choice([
                "I'm sorry that work is causing you sadness. It can be really tough sometimes. 💙",
                "Work can certainly bring its challenges. Would you like to talk more about what's happening there with your job?",
                "It sounds like work is a source of sadness for you. How does that impact your day-to-day?"
            ])
        elif "alone" in user_text_lower or "lonely" in user_text_lower:
            return random.choice([
                "Feeling alone can be incredibly difficult, and it's okay to acknowledge that sadness. 🫂",
                "I hear that you're feeling lonely. Sometimes connecting, even virtually, can help. I'm here to listen.",
                "It sounds like you're experiencing loneliness. What does 'feeling alone' mean to you in this moment?"
            ])
        elif any(topic in user_text_lower for topic in ["friend", "family", "partner", "relationship"]) or \
             any(topic in current_session_topics for topic in ["friend", "family", "partner", "relationship"]):
             return random.choice([
                "I'm sorry to hear your relationships are causing sadness. That can be incredibly painful. 💔",
                "Relationships can be complex. Would you like to explore what's making you sad about this connection?",
                "It sounds like there's some sadness related to a relationship. I'm here to listen without judgment."
            ])
        elif "future" in user_text_lower or "plans" in current_session_topics:
            return random.choice([
                "It sounds like uncertainty about the future is bringing on some sadness. What are your hopes or fears for what's ahead?",
                "Feeling sad about future plans is understandable. What aspects are most concerning you right now?"
            ])
        else:
            return random.choice(emotion_reply_bank.get(emotion))

    elif emotion == "anger":
        if any(topic in user_text_lower for topic in ["person", "he", "she", "they", "boss"]) or \
           any(topic in current_session_topics for topic in ["person", "boss", "colleagues"]):
            return random.choice([
                "It sounds like someone specific has really upset you. Would you like to vent about it?",
                "Anger towards a person can be consuming. What about their actions is making you so angry?",
                "I hear your anger towards someone specific. What would it feel like to express that more?"
            ])
        elif "unfair" in user_text_lower or "injustice" in user_text_lower:
            return random.choice([
                "It sounds like you're feeling angry about an injustice. That's a valid feeling. 😡",
                "Anger in the face of unfairness can be powerful. What makes this situation feel so unjust to you?"
            ])
        else:
            return random.choice(emotion_reply_bank.get(emotion))

    # --- Priority 4: General "Topic Awareness" (even if primary emotion is different) ---
    # (This section remains largely the same as the previous update)
    if "stress" in current_session_topics and emotion != "relief":
        return random.choice([
            "It sounds like stress is still a significant theme for you. How are you coping with the pressure?",
            "Remember you mentioned stress earlier. Is this current feeling related to that ongoing stress?",
            "It seems like stress is a recurring theme. What's one small thing you could do to ease some pressure today?"
        ])

    # --- Priority 5: Default behavior (if no specific context or previous emotion matched) ---
    # (This section remains largely the same as the previous update)
    if emotion == "desire" and ("food" in user_text_lower or "eat" in user_text_lower):
        return random.choice([
            "It sounds like you're feeling a strong urge for food right now. Are you feeling hungry, or is there something else on your mind?",
            "Sometimes when we're feeling a certain way, our bodies crave things. What's making you want to eat right now?",
            "That's a very human need! How is that desire connected to what you were just sharing, if at all?"
        ])
    elif emotion == "curiosity" and user_text_lower == "huh?":
         return random.choice([
            "Sorry, could you clarify what you meant by 'huh?' I want to make sure I understand.",
            "My apologies if my last response was unclear. What part confused you?",
            "It seems like I didn't quite hit the mark. What were you hoping for me to say?"
        ])

    # Fallback to general emotion replies if no specific rule matched
    return random.choice(emotion_reply_bank.get(emotion, ["I’m here to talk, whenever you’re ready."]))


# -------------------- JOURNAL RESPONSE FUNCTION (UPDATED for new generate_reply_with_context signature) --------------------
def journal_response(text, top_k=2, threshold=0.2):
    """
    Analyzes user text, detects emotions, and generates a counseling-like reply,
    considering the current session's topics and last strong emotion.
    """
    global last_strong_emotion, last_strong_emotion_turn_count

    # 1. Update session topics based on current user input
    update_session_topics(text)

    emotions_with_scores = predict_emotions(text)

    sorted_emotions = sorted(
        [(em, score) for em, score in emotions_with_scores.items() if score >= threshold],
        key=lambda x: x[1],
        reverse=True
    )

    top_emotions = [em for em, _ in sorted_emotions[:top_k]]

    # Update last_strong_emotion logic
    current_primary_emotion = None
    if sorted_emotions and sorted_emotions[0][1] >= LAST_STRONG_EMOTION_THRESHOLD:
        current_primary_emotion = sorted_emotions[0][0]
        if current_primary_emotion != "neutral":
            last_strong_emotion = current_primary_emotion
            last_strong_emotion_turn_count = 0
    else:
        if last_strong_emotion:
            last_strong_emotion_turn_count += 1
            if last_strong_emotion_turn_count >= LAST_STRONG_EMOTION_DECAY:
                last_strong_emotion = None
                last_strong_emotion_turn_count = 0


    replies = []
    # Pass all detected emotions' names to generate_reply_with_context for comprehensive check
    all_detected_emotion_names = list(emotions_with_scores.keys())

    if not top_emotions:
        chosen_emotion = last_strong_emotion if last_strong_emotion else "neutral"
        replies.append(generate_reply_with_context(chosen_emotion, text, current_session_topics, last_strong_emotion, all_detected_emotion_names))
        emoji = emoji_map.get(chosen_emotion, "")
    else:
        primary_detected_emotion = top_emotions[0]
        replies.append(generate_reply_with_context(primary_detected_emotion, text, current_session_topics, last_strong_emotion, all_detected_emotion_names))
        emoji = emoji_map.get(primary_detected_emotion, "")

        if len(top_emotions) > 1 and top_emotions[1] != primary_detected_emotion:
            if primary_detected_emotion != "neutral" and top_emotions[1] == "neutral":
                pass
            else:
                if top_emotions[1] != "neutral":
                    # For secondary emotions, we can just pass the same info
                    replies.append(generate_reply_with_context(top_emotions[1], text, current_session_topics, last_strong_emotion, all_detected_emotion_names))


    final_reply = f"{emoji} " + "\n".join(replies)

    return {
        "user_input": text,
        "detected_emotions": emotions_with_scores,
        "top_emotions": top_emotions,
        "reply": final_reply
    }


In [ ]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')